In [ ]:
import numpy as np
import os
from pprint import pprint
from bisect import bisect
from scipy.interpolate import interp1d
from scipy.spatial.transform import Slerp, Rotation

In [ ]:
head = "#timestamp tx ty tz qx qy qz qw\n"

with open("/media/prakhar/BIG_BAG/Capstone/home1-1_rgbd/odom.txt","r") as fp:
    data_odom = np.array([np.array([float(zz) for zz in l[:-1].split(" ")[:8]]) for l in fp.readlines() if "#" not in l])

with open("/media/prakhar/BIG_BAG/Capstone/home1-1_rgbd/rgb.txt","r") as fp:
    time_stamps_rgb = np.array([np.array(float(l[:-1].split(" ")[0])) for l in fp.readlines() if "#" not in l])

# with open("/media/prakhar/BIG_BAG/Capstone/home1-1_rgbd/odom_pose.txt","w") as fp:
#     fp.write(head)
#     for l in data:
#         fp.write(l)

In [ ]:
T1 = np.array([
    [1.0,0.0,0],
    [0.0,0,-1.0],
    [0.0,1.0,0.0]
])
# T2 = np.array([
#     [1.0,0,0],
#     [0,0,-1],
#     [0,1,0]
# ])
# T2 = np.array([
#     [0.0,-1.0,0],
#     [1.0,0,0],
#     [0,0,1.0]
# ])

# T2 = T2@T1
# print(T, T.dtype)
theta = np.pi/2
ct = np.cos(theta)
st = np.sin(theta)
Rytheta = np.array([
    [ct,0.0,st],
    [0.0,1.0,0.0],
    [-st,0.0,ct]
])

In [ ]:
time_stamps_rgb[-1] - time_stamps_rgb[0], time_stamps_rgb[0], time_stamps_rgb[-7]

In [ ]:
data_odom[-1,0] - data_odom[0,0], data_odom[0,0], data_odom[-1,0]

In [ ]:
ind_max = bisect(time_stamps_rgb, data_odom[-1,0])

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
#create interpolators
coord_interpolator = interp1d(data_odom[:,0], data_odom[:,1:4], axis=0)
rotations = Rotation(data_odom[:,4:])
orien_interpolator = Slerp(data_odom[:,0], rotations)

#interpolate at precise timesteps
trans = coord_interpolator(time_stamps_rgb[:ind_max])
rots = orien_interpolator(time_stamps_rgb[:ind_max])
quats = rots.as_quat()

In [ ]:
trans_kitti = (T1@trans.T).T
print(trans_kitti.shape)
rots_dcm = rots.as_dcm()
rots_dcm_kitti = []
for i in range(rots_dcm.shape[0]):
    rotdcm = rots_dcm[i]
    rotm_kitti = Rytheta@T1@rotdcm@T1.T
    rots_dcm_kitti.append(rotm_kitti)
rots_dcm_kitti = np.array(rots_dcm_kitti)
print(rots_dcm_kitti.shape)
quats_kitti = Rotation.from_dcm(rots_dcm_kitti).as_quat()
print(quats_kitti.shape)

In [ ]:
with open("/media/prakhar/BIG_BAG/Capstone/home1-1_rgbd/odom_pose_aligned_kitti.txt","w") as fp:
    fp.write("#timestamp tx ty tz qx qy qz qw\n")
    for i in range(len(trans)):
        time = str(time_stamps_rgb[i])
        tr = " ".join([str(t) for t in trans_kitti[i]])
        qu = " ".join([str(q) for q in quats_kitti[i]])
#         print(tr, qu)
        string = time + " " + tr + " " + qu + "\n"
#         print(string)
        fp.write(string)

In [ ]:
quats.shape

In [ ]:
pose = np.array([[-0.3096456800225544,-0.9508520141659108,0,3.004341876402727],[0.9508520141659108,-0.3096456800225544,0,-3.846301833905982],[0,0,1,0],[0,0,0,1]])

                


In [ ]:
np.linalg.det(pose[:3,:3])

In [ ]:
quats[0]

In [ ]:
rotations[0].as_dcm()

In [ ]:
mats[0]

In [ ]:
#frame 1
curr_co1 = np.array([[-0.2472927550798112,0.9689408100008156,0,4.579310547681612],[-0.9689408100008156,-0.2472927550798112,0,1.962745705102641],[0,0,1,0],[0,0,0,1]])
prev_oc1 = np.array([[-0.2477349348950084,-0.9688278495339435,0,3.03421473503],[0.9688278495339435,-0.2477349348950084,0,-3.95170807838],[0,0,1,0],[0,0,0,1]])
prev_cw1 = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
curr_cw1 = np.array([[0.9999998958584718,-0.0004563803733710915,0,0],[0.0004563803733711014,0.9999998958584717,0,0],[0,0,1,0],[0,0,0,1]])
#frame 1 
curr_co=np.array([[-0.2472927550798112,0.9689408100008156,0,4.579310547681612],[-0.9689408100008156,-0.2472927550798112,0,1.962745705102641],[0,0,1,0],[0,0,0,1]])
prev_oc=np.array([[-0.2477349348950084,-0.9688278495339435,0,3.03421473503],[0.9688278495339435,-0.2477349348950084,0,-3.95170807838],[0,0,1,0],[0,0,0,1]])
prev_cw=np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
curr_cw=np.array([[0.9999998958584718,-0.0004563803733710915,0,0],[0.0004563803733711014,0.9999998958584717,0,0],[0,0,1,0],[0,0,0,1]])
curr_cw_postoptim=np.array([[0.9999998958584718,-0.0004563803733710915,0,0],[0.0004563803733711014,0.9999998958584717,0,0],[0,0,1,0],[0,0,0,1]])
#frame 2
curr_co=np.array([[-0.2470575672139561,0.969000804170111,0,4.577082710922297],[-0.969000804170111,-0.2470575672139561,0,1.963857149610757],[0,0,1,0],[0,0,0,1]])
prev_oc=np.array([[-0.2472927550798112,-0.9689408100008156,0,3.03421473503],[0.9689408100008156,-0.2472927550798112,0,-3.95170807838],[0,0,1,0],[0,0,0,1]])
prev_cw=np.array([[0.9999998958584715,-0.0004563803733710813,0,0],[0.0004563803733711113,0.9999998958584712,0,0],[0,0,1,0],[0,0,0,1]])
curr_cw=np.array([[0.9999997556298589,-0.0006990995793084267,0,-0.001751305714697615],[0.0006990995793084735,0.9999997556298587,0,1.552618678068995e-08],[0,0,1,0],[0,0,0,1]])
curr_cw_postoptim=np.array([[0.9999997556298589,-0.0006990995793084267,0,-0.001751305714697615],[0.0006990995793084735,0.9999997556298587,0,1.552618678068995e-08],[0,0,1,0],[0,0,0,1]])

#frame 3 
curr_co=np.array([[-0.2469693717556598,0.9690232863118485,0,4.574574140826023],[-0.9690232863118485,-0.2469693717556598,0,1.964273537170282],[0,0,1,0],[0,0,0,1]])
prev_oc=np.array([[-0.2470575672139561,-0.969000804170111,0,3.033782076745566],[0.969000804170111,-0.2470575672139561,0,-3.950011057898249],[0,0,1,0],[0,0,0,1]])
prev_cw=np.array([[0.9999997556298581,-0.0006990995793083791,0,-0.001751305714697615],[0.0006990995793085197,0.9999997556298574,0,1.552618678068996e-08],[0,0,1,0],[0,0,0,1]])
curr_cw=np.array([[0.9999996878587718,-0.0007901154074278755,0,-0.004081114712969996],[0.0007901154074280313,0.9999996878587711,0,-3.352650491251408e-07],[0,0,1,0],[0,0,0,1]])
curr_cw_postoptim=np.array([[0.9999996878587718,-0.0007901154074278755,0,-0.004081114712969996],[0.0007901154074280313,0.9999996878587711,0,-3.352650491251408e-07],[0,0,1,0],[0,0,0,1]])

#frame 4 
curr_co=np.array([[-0.2471125272345491,0.968986789839652,0,4.572536020830874],[-0.968986789839652,-0.2471125272345491,0,1.963597845992639],[0,0,1,0],[0,0,0,1]])
prev_oc=np.array([[-0.2469693717556598,-0.9690232863118485,0,3.033206499813635],[0.9690232863118485,-0.2469693717556598,0,-3.947753465989221],[0,0,1,0],[0,0,0,1]])
prev_cw=np.array([[0.9999996878587699,-0.0007901154074277176,0,-0.004081114712969996],[0.0007901154074281851,0.9999996878587679,0,-3.352650491251409e-07],[0,0,1,0],[0,0,0,1]])
curr_cw=np.array([[0.9999997936733388,-0.0006423809415028752,0,-0.006409375767553446],[0.0006423809415033488,0.9999997936733368,0,4.203534141921818e-07],[0,0,1,0],[0,0,0,1]])
curr_cw_postoptim=np.array([[0.9999997936733388,-0.0006423809415028752,0,-0.006409375767553446],[0.0006423809415033488,0.9999997936733368,0,4.203534141921818e-07],[0,0,1,0],[0,0,0,1]])

#frame 5 
curr_co=np.array([[-0.2473597109167724,0.9689237190899865,0,4.572464880361144],[-0.9689237190899865,-0.2473597109167724,0,1.962431496498624],[0,0,1,0],[0,0,0,1]])
prev_oc=np.array([[-0.2471125272345491,-0.968986789839652,0,3.032631305302989],[0.968986789839652,-0.2471125272345491,0,-3.945497374055528],[0,0,1,0],[0,0,0,1]])
prev_cw=np.array([[0.9999997936733329,-0.0006423809415023966,0,-0.006409375767553446],[0.0006423809415038174,0.999999793673327,0,4.203534141921826e-07],[0,0,1,0],[0,0,0,1]])
curr_cw=np.array([[0.9999999250079884,-0.0003872776781519474,0,-0.006981287417182163],[0.0003872776781533466,0.9999999250079825,0,2.238811268990866e-06],[0,0,1,0],[0,0,0,1]])
curr_cw_postoptim=np.array([[0.9999999250079884,-0.0003872776781519474,0,-0.006981287417182163],[0.0003872776781533466,0.9999999250079825,0,2.238811268990866e-06],[0,0,1,0],[0,0,0,1]])

#frame 6 
curr_co=np.array([[-0.2476408601911948,0.9688519001187771,0,4.573034140967205],[-0.9688519001187771,-0.2476408601911948,0,1.961104588270142],[0,0,1,0],[0,0,0,1]])
prev_oc=np.array([[-0.2473597109167724,-0.9689237190899865,0,3.03249001503],[0.9689237190899865,-0.2473597109167724,0,-3.94494318962],[0,0,1,0],[0,0,0,1]])
prev_cw=np.array([[0.9999999250079709,-0.0003872776781505392,0,-0.006981287417182164],[0.0003872776781547366,0.9999999250079532,0,2.238811268990866e-06],[0,0,1,0],[0,0,0,1]])
curr_cw=np.array([[0.9999999952857334,-9.710036305712094e-05,0,-0.006981286473607658],[9.710036306131836e-05,0.9999999952857159,0,4.264622451437246e-06],[0,0,1,0],[0,0,0,1]])
curr_cw_postoptim=np.array([[0.9999999952857334,-9.710036305712094e-05,0,-0.006981286473607658],[9.710036306131836e-05,0.9999999952857159,0,4.264622451437246e-06],[0,0,1,0],[0,0,0,1]])

#frame 7 
curr_co=np.array([[-0.2479221032657,0.9687799702266306,0,4.573603248444285],[-0.9687799702266306,-0.2479221032657,0,1.959776973639028],[0,0,1,0],[0,0,0,1]])
prev_oc=np.array([[-0.2476408601911948,-0.9688519001187771,0,3.03249001503],[0.9688519001187771,-0.2476408601911948,0,-3.94494318962],[0,0,1,0],[0,0,0,1]])
prev_cw=np.array([[0.9999999952856806,-9.71003630529167e-05,0,-0.006981286473607658],[9.710036306550896e-05,0.9999999952856283,0,4.264622451437246e-06],[0,0,1,0],[0,0,0,1]])
curr_cw=np.array([[0.9999999813377072,0.0001931953071641074,0,-0.006981284941444227],[-0.0001931953071515354,0.9999999813376548,0,6.291259488568115e-06],[0,0,1,0],[0,0,0,1]])
curr_cw_postoptim=np.array([[0.9999999813377072,0.0001931953071641074,0,-0.006981284941444227],[-0.0001931953071515354,0.9999999813376548,0,6.291259488568115e-06],[0,0,1,0],[0,0,0,1]])


#frame 8 
curr_co=np.array([[-0.2481254474322159,0.968727909341195,0,4.572251518108446],[-0.968727909341195,-0.2481254474322159,0,1.958816891211766],[0,0,1,0],[0,0,0,1]])
prev_oc=np.array([[-0.2479221032657,-0.9687799702266306,0,3.03249001503],[0.9687799702266306,-0.2479221032657,0,-3.94494318962],[0,0,1,0],[0,0,0,1]])
prev_cw=np.array([[0.9999999813375485,0.0001931953071766386,0,-0.006981284941444227],[-0.0001931953071389226,0.9999999813373914,0,6.291259488568115e-06],[0,0,1,0],[0,0,0,1]])
curr_cw=np.array([[0.9999999187557199,0.0004030980988014113,0,-0.008744275722064612],[-0.0004030980987637316,0.9999999187555628,0,7.729523445520093e-06],[0,0,1,0],[0,0,0,1]])
curr_cw_postoptim=np.array([[0.9999999187557199,0.0004030980988014113,0,-0.008744275722064612],[-0.0004030980987637316,0.9999999187555628,0,7.729523445520093e-06],[0,0,1,0],[0,0,0,1]])

#frame 9 


#frame 10 
#frame 11 
#frame 12 
#frame 13 
#frame 14
#frame 15
#frame 16
#frame 17






curr_cw_rec = curr_co@prev_oc1@prev_cw1
print(curr_cw)
print(curr_cw_rec)

In [ ]:
curr_co=np.array([[-0.2729258772317533,0.9620350646090188,0,4.572144521523372],[-0.9620350646090188,-0.2729258772317533,0,1.841561782787102],[0,0,1,0],[0,0,0,1]])
prev_oc=np.array([[-0.2703833037146914,-0.9627527559411757,0,3.020137243152615],[0.9627527559411757,-0.2703833037146914,0,-3.898195265054517],[0,0,1,0],[0,0,0,1]])
prev_cw=np.array([[0.9972435714755002,0.02354739552595232,0,-0.0553259272690979],[-0.02319236181519389,0.9955974437060024,0,0.0009158271563182595],[0,0,1,0],[0,0,0,1]])
curr_cw=np.array([[0.9971788188173132,0.02617760355477675,0,-0.05765293324448762],[-0.02582692002324823,0.9955317588214413,0,0.001064207990545391],[0,0,1,0],[0,0,0,1]])
# curr_cw_postoptim=np.array([[0.9971788188173132,0.02617760355477675,0,-0.05765293324448762],[-0.02582692002324823,0.9955317588214413,0,0.001064207990545391],[0,0,1,0],[0,0,0,1]])

curr_co2=np.array([[-0.2754074554502306,0.9613275890571483,0,4.574551826124208],[-0.9613275890571483,-0.2754074554502306,0,1.829761131471451],[0,0,1,0],[0,0,0,1]])
prev_oc2=np.array([[-0.2729258772317533,-0.9620350646090188,0,3.019503563052208],[0.9620350646090188,-0.2729258772317533,0,-3.895953485121863],[0,0,1,0],[0,0,0,1]])
prev_cw2=np.array([[0.9922357268852574,0.02635325526606056,0,-0.05765293324448762],[-0.02530814797869082,0.9873271576207754,0,0.001064207990545391],[0,0,1,0],[0,0,0,1]])
curr_cw2=np.array([[0.9921671168566281,0.02890091949118303,0,-0.05997953298670251],[-0.02786848201883031,0.9872558670837074,0,0.001216662112142389],[0,0,1,0],[0,0,0,1]])
# curr_cw_postoptim2=np.array([[0.9921671168566281,0.02890091949118303,0,-0.05997953298670251],[-0.02786848201883031,0.9872558670837074,0,0.001216662112142389],[0,0,1,0],

In [ ]:
curr_co

In [ ]:
np.linalg.inv(prev_oc2)

In [ ]:
curr_cw

In [ ]:
prev_cw2

In [ ]:
trans[0]

In [ ]:
trans[100]

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.figure()
plt.plot(trans[:200,0], trans[:200,1])
plt.show()

In [ ]:
rots_dcm[200]

In [ ]:
M1 = np.eye(4)
print(M1)
M1o = np.eye(4)
M1o[:3,:3] = rots_dcm[0]
M1o[:3,3] = trans[0]
print(M1o)
M2o = np.eye(4)
M2o[:3,:3] = rots_dcm[200]
M2o[:3,3] = trans[200]
print(M2o)

In [ ]:
M2 = np.linalg.inv(M2o)@M1o@M1
print(M2)

In [ ]:
np.arcsin(0.94)*180/np.pi

In [ ]:
print(T1)
print(T1.T)
print(np.linalg.inv(T1))

In [ ]:
import cv2

In [ ]:
imd = cv2.imread("/media/prakhar/BIG_BAG/Capstone/home1-1/depth/1560000002.527835.png", cv2.IMREAD_UNCHANGED)

In [ ]:
imda = cv2.imread("/media/prakhar/BIG_BAG/Capstone/home1-1_rgbd/aligned_depth/1560000002.527835.png", cv2.IMREAD_UNCHANGED)

In [ ]:
imda.shape, imda.dtype

In [ ]:
np.max(imda)

In [ ]:
|